# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание столбцов:

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import numpy as np

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
display(df_0.head())
display(df_1.head())
display(df_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Т.к. мы будем использовать данные с признаками для прогнозирования, то столбцы с id нам не нужны, удалим их из датасетов.**

In [7]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

In [8]:
display(df_0.head())
display(df_1.head())
display(df_2.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


## Обучение и проверка модели

Подготовим данные. Изначально выделим признаки и целевой признак, далее разделим на обучающую и валидационную выборки в соотношении 75:25.

In [9]:
target_0 = df_0['product']
features_0 = df_0.drop(['product'] , axis=1)
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

#Обучим модель линейной регрессии, получим предсказания, средний запас предсказанного сырья и RMSE модели.
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predicted_valid_0 = model.predict(features_valid_0)
mse_0 = mean_squared_error(target_valid_0, predicted_valid_0)
predictions_valid_0 = pd.Series(predicted_valid_0)

print('Для региона 0 средний запас предсказанного сырья составил', predictions_valid_0.mean(), 'а RMSE модели', mse_0 ** 0.5)
print('Для региона 0 истинный средний запас сырья равен', target_valid_0.mean())

Для региона 0 средний запас предсказанного сырья составил 92.59256778438038 а RMSE модели 37.5794217150813
Для региона 0 истинный средний запас сырья равен 92.07859674082927


In [10]:
target_1 = df_1['product']
features_1 = df_1.drop(['product'] , axis=1)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

#Обучим модель линейной регрессии, получим предсказания, средний запас предсказанного сырья и RMSE модели.
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predicted_valid_1 = model.predict(features_valid_1)
mse_1 = mean_squared_error(target_valid_1, predicted_valid_1)
predictions_valid_1 = pd.Series(predicted_valid_1)

print('Для региона 1 средний запас предсказанного сырья составил', predictions_valid_1.mean(), 'а RMSE модели', mse_1 ** 0.5)
print('Для региона 1 истинный средний запас сырья равен', target_valid_1.mean())

Для региона 1 средний запас предсказанного сырья составил 68.728546895446 а RMSE модели 0.893099286775616
Для региона 1 истинный средний запас сырья равен 68.72313602435997


In [11]:
target_2 = df_2['product']
features_2 = df_2.drop(['product'] , axis=1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

#Обучим модель линейной регрессии, получим предсказания, средний запас предсказанного сырья и RMSE модели.
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predicted_valid_2 = model.predict(features_valid_2)
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
predictions_valid_2 = pd.Series(predicted_valid_2)

print('Для региона 2 средний запас предсказанного сырья составил', predictions_valid_2.mean(), 'а RMSE модели', mse_2 ** 0.5)
print('Для региона 2 истинный средний запас сырья равен', target_valid_2.mean())

Для региона 2 средний запас предсказанного сырья составил 94.96504596800489 а RMSE модели 40.02970873393434
Для региона 2 истинный средний запас сырья равен 94.88423280885438


По результату проведенного моделирования можно сказать следующее:

1. Все 3 модели предсказывают близкие к реальным значения, так как истинные средние запасы отличаются от предсказанных для каждой модели всего на несколько сотых.
2. Наилучший прогноз у модели для региона 1. Предсказательные способности моделей для  регионов 0 и 2 примерно одинаковые.

## Подготовка к расчёту прибыли

### Сохранение ключевых переменных

In [12]:
explore_count = 500 # количество точек для исследования
mining_count = 200 # количество точек для разработки
mining_budget = 10000 # бюджет на разработку скважин в регионе в млн. руб.
profit_per_barrel = 0.45 # доход с 1 тыс баррель в млн. руб
loss_probability = 0.025 # максимальная вероятность убытков
bootstrap_samples = 1000

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [13]:
budget_per_borehole = mining_budget / mining_count #посчитаем затраты на разработку одной скважины
#посчитаем какой объем сырья необходим для безубыточной разработки 1 скважины в тыс. баррелей
required_volume = budget_per_borehole / profit_per_barrel
print('Необходимый объем сырья для безубыточной разработки 1 скважины в тыс. баррелей', required_volume)

Необходимый объем сырья для безубыточной разработки 1 скважины в тыс. баррелей 111.11111111111111


Необходимый объем сырья для того, чтобы разработка была окупаемой - 111 тысяч баррелей. Средние значения запасов сырья для всех трех регионов - ниже необходимого объема. При осуществлении разработки скважин, выбранных случайным образом, есть высокая вероятность не окупить затраченный бюджет. 

## Расчёт прибыли и рисков 

### Расчитаем прибыль 200 скважин с самыми высокими предсказаниями модели

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберем скважины с максимальными значениями предсказаний.
- Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитаем прибыль для полученного объёма сырья.

In [14]:
def income(target_valid, predictions_valid):
    predictions_result = predictions_valid.sort_values(ascending=False).head(200) #выбираем 200 скважин с максимальным объемом запасов сырья
    target_result = target_valid.reset_index(drop=True)[predictions_result.index] #находим реальные значения для отобранных 200 скважин
    income = target_result.sum() * profit_per_barrel #доход с 200 скважин млн.руб.
    return income

In [15]:
print('Прибыль с 200 выбранных по предсказаниям модели скважин составит:')
print('для региона 0 - ', income(target_valid_0, predictions_valid_0) - mining_budget,'млн. руб.')

Прибыль с 200 выбранных по предсказаниям модели скважин составит:
для региона 0 -  3320.826043139852 млн. руб.


In [16]:
print('Прибыль с 200 выбранных по предсказаниям модели скважин составит:')
print('для региона 1 - ', income(target_valid_1, predictions_valid_1) - mining_budget,'млн. руб.')

Прибыль с 200 выбранных по предсказаниям модели скважин составит:
для региона 1 -  2415.086696681512 млн. руб.


In [17]:
print('Прибыль с 200 выбранных по предсказаниям модели скважин составит:')
print('для региона 2 - ', income(target_valid_2, predictions_valid_2) - mining_budget,'млн. руб.')

Прибыль с 200 выбранных по предсказаниям модели скважин составит:
для региона 2 -  2710.3499635998323 млн. руб.


Получаются очень высокие значения прибыли для каждого региона от 2.4 млрд руб. до 3.3 млрд руб. Но это значения полученные  при  проведении геологоразведки по 25000 точкам, а согласно требованиям задания, мы можем провести геологоразведку только по 500 точкам.

### Расчитаем прибыль и риски для каждого региона для 500 случайных скважин из 25000.

Функция похожа на предыдущую, только учитывается количество скважин по которым ведется поиск наибольших значений запасов.

In [18]:
def revenue (target_valid, predictions_valid, mining_count):
    predictions_result = predictions_valid.sort_values(ascending=False) #сортируем скважины по значениям предсказаний
    #определяем верные значения для отобранных случайным образом скважин и берем только нужное количество
    target_result = target_valid[predictions_result.index][:mining_count]
    income = target_result.sum() * profit_per_barrel #доход
    return income

- Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [19]:
#создадим функцию для применения для каждого региона
def income_calc (predictions_valid, target_valid, bootstrap_samples, explore_count):
    state = np.random.RandomState(12345)
    count = 0 #счетчик для подсчета вероятности убытков в регионе
    values = [] #все значения выручек для 1000 выборок
    for i in range(bootstrap_samples): #создаем 1000 выборок
        #у целевого признака валидационной выборки удаляем индексы и случайно выбираем нужное количество скважин
        target_subsample = target_valid.reset_index(drop=True).sample(explore_count, replace=True, random_state=state)
        #из предсказаний выбираем строки соответствующие отобранным строкам в целевом признаке
        prob_subsample = predictions_valid[target_subsample.index]
        #считаем выручку для отобранных строк, для нужного количества скважин
        rev = revenue(target_subsample, prob_subsample, mining_count) - mining_budget
        #добавляем выручку в список выручек
        values.append(rev)
        #проверяем не является ли выборка убыточной, если да, то увеличиваем счетчик
        if rev < 0:
            count +=1
    values = pd.Series(values)
    #посчитаем и выведем среднюю выручку
    mean = values.mean()
    print("Средняя выручка:", mean, 'млн рублей')
    #посчитаем и выведем 95-% доверительный интервал
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), np.std(values, ddof=1))
    print("95%-ый доверительный интервал:", confidence_interval)
    #вероятноть убытков должна быть ниже 2.5%
    pvalue = count / bootstrap_samples
    if pvalue < loss_probability:
        print('Вероятность убытков равна', pvalue * 100, '% и является меньше допустимой, регион подходит по критериям')
    else:
        print('Вероятность убытков равна', pvalue * 100, '% и является больше допустимой, регион не подходит по критериям')    

In [20]:
print('Для "0" региона при случайном выборе 500 скважин получаются следующие показатели')
income_calc(predictions_valid_0, target_valid_0, bootstrap_samples, explore_count)

Для "0" региона при случайном выборе 500 скважин получаются следующие показатели
Средняя выручка: 425.9385269105927 млн рублей
95%-ый доверительный интервал: (-118.1730815867329, 970.0501354079183)
Вероятность убытков равна 6.0 % и является больше допустимой, регион не подходит по критериям


In [21]:
print('Для "1" региона при случайном выборе 500 скважин получаются следующие показатели')
income_calc(predictions_valid_1, target_valid_1, bootstrap_samples, explore_count)

Для "1" региона при случайном выборе 500 скважин получаются следующие показатели
Средняя выручка: 515.2227734432902 млн рублей
95%-ый доверительный интервал: (85.1119884768645, 945.3335584097158)
Вероятность убытков равна 1.0 % и является меньше допустимой, регион подходит по критериям


In [22]:
print('Для "2" региона при случайном выборе 500 скважин получаются следующие показатели')
income_calc(predictions_valid_2, target_valid_2, bootstrap_samples, explore_count)

Для "2" региона при случайном выборе 500 скважин получаются следующие показатели
Средняя выручка: 435.00836278275585 млн рублей
95%-ый доверительный интервал: (-120.12349557730545, 990.1402211428172)
Вероятность убытков равна 6.4 % и является больше допустимой, регион не подходит по критериям


**Можно сделать следующие выводы**

При случайном выборе 500 скважин для проведения геологоразведки, потенциальная прибыль падает в несколько раз по сравнению с той, что была бы если бы мы проводили разведку для всего объема скважин.
Исходя из расчитанной средней выручки - все три региона прибыльные.
0-й и 2-й регионы имеют вероятность убытков выше допустимой по заданию, поэтому не подходят.
1-й регион лидирует по всем показателям. Он обладает наибольшей средней прибылью, и единственный соответствует условию по минимальной вероятности убытков. Судя по данным для дальнейшей разработки заказчику предлагается принять только 1-й регион.